In [1]:
import teradatasql as td
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from getpass import getpass
username=input('Username: ')
pswd=getpass('Passwd: ')
con = td.connect(None, host='tdprod.kb.cz', tmode='TERA', logmech='LDAP', user=username, password=pswd)
username='E_EKOUA2'
pswd='Etl-21IE,86at'
del(pswd)
del(username)

Username:  e_jkossa
Passwd:  ········


#### Permet de recupérer la dernière date disponible de façon journalière 

In [2]:
sql_string2 = f"""
select max(date_valid) as dd_date
from ibfs_ben_m_marketing_vu.VF_Dd_Agreement_Detail a
where a.Segmentation_Code_IBFS like '101%'
and a.party_status='C'
and a.mandatory_product_flag='Y'
;""" 
cursor = con.cursor()
result_Set1 = cursor.execute(sql_string2)
result_Set1 = pd.read_sql(sql_string2, con)
ddt = result_Set1.iloc[0]
last_date_journaliere = ddt['dd_date'].strftime("%Y-%m-%d")

In [3]:
from datetime import datetime
import glob
date_jour = datetime.now()
num=date_jour.strftime("%U")

#### Permet de recupérer la dernière date disponible de façon mensuel 

In [4]:
sql_string3 = f"""
select max(date_valid) as dm_date
from ibfs_ben_m_marketing_vu.VF_Dm_Agreement_Detail a
where a.Segmentation_Code_IBFS like '101%'
and a.party_status='C'
and a.mandatory_product_flag='Y'
;""" 

cursor = con.cursor()
result_Set2 = cursor.execute(sql_string3)
result_Set2 = pd.read_sql(sql_string3, con)
dmt = result_Set2.iloc[0]
last_date_mensuel = dmt['dm_date'].strftime("%Y-%m-%d")

### OC1

In [5]:
# Example of parametrized query
dd = last_date_journaliere
dt = last_date_mensuel
liste_1 = []
Entity =['BEN']
for i in Entity :
    sql_string_i = f""" 
select distinct t2.Identifiant , t2.Nom_client,t2.segmentation_code_description ,
t2.profile_code_description,t2.Code_gestionnaire,t2.gestionnaire,t2.agence,
t2.age,t2.activity_flag,t2.profession_group_desc_1,t2.Montant_du_Credit,t2.Revenu,t2.contact_text,
t2.product_name,t2.contract_date,t2.maturity_date,t2.type_
from

(select distinct t1.Identifiant , t1.Nom_client,t1.segmentation_code_description ,
t1.profile_code_description,t1.Code_gestionnaire,t1.gestionnaire,t1.agence,
t1.age,t1.activity_flag,t1.profession_group_desc_1,t1.Montant_du_Credit,t1.Revenu,t1.contact_text,
t1.product_name,t1.contract_date,t1.maturity_date,t1.type_
from (

select distinct a.party_id as Identifiant , a.party_short_name as Nom_client,a.segmentation_code_description ,
a.profile_code_description,a.officer_id as Code_gestionnaire,a.officer_name as gestionnaire,a.org_unit_name_4 as agence,
to_number(substr(a.Age_Group_Description,1,2)) as age,a.activity_flag,profession_group_desc_1,a.sale_amt_lc as Montant_du_Credit,a.marketing_income_amt as Revenu,a.contact_text,
a.product_name,a.contract_date,a.maturity_date,'Credits echus et non renouvelles sur les 6 derniers mois' as type_
from ibfs_{i}_m_marketing_vu.VF_Dm_Agreement_Detail a
where a.Segmentation_Code_IBFS like '101%'
and org_structure_type = 'C'
and a.party_status='C'
and a.mandatory_product_flag='Y'
and a.Product_Group_Code_3= '1RECRCC'
and a.date_valid = '{dt}'
and a.sale_amt_lc >0
and a.maturity_date between  add_months('{dt}', -6) and  '{dt}'  
and not (a.profile_code_description like  '%PERSONNEL%'  or a.profile_code_description like  '%DECEDE%')
and age between 22 and 64
and revenu > 80000
union 

select distinct a.party_id as Identifiant , a.party_short_name as Nom_client,a.segmentation_code_description ,
a.profile_code_description,a.officer_id as Code_gestionnaire,a.officer_name as gestionnaire,a.org_unit_name_4 as agence,
to_number(substr(a.Age_Group_Description,1,2)) as age,
a.activity_flag,b.profession_group_desc_1,a.sale_amt_lc as Montant_du_Credit,
a.marketing_income_amt as Revenu,a.contact_text,a.product_name,a.contract_date,b.maturity_date,'Credits arrivant a echeance dans – de 3 mois' as type_
from ibfs_{i}_m_marketing_vu.VF_Dd_Agreement_Detail a
left join ibfs_{i}_m_marketing_vu.VF_Dm_Agreement_Detail b
on a.agreement_id = b.agreement_id
where a.Segmentation_Code_IBFS like '101%'
and a.party_status='C'
and a.mandatory_product_flag='Y'
and a.Product_Group_Code_3='1RECRCC'
and a.product_status='Y'
and  a.date_valid = '{dd}'
and b.maturity_date between '{dd}' and add_months('{dd}', 3)
and not (a.profile_code_description like  '%PERSONNEL%')
and age between 22 and 64
and revenu > 80000
) as t1
where t1.Identifiant not in (
select distinct party_id 
from  ibfs_{i}_m_risk_vu.dm_Agreement 
where Provi_Stage_Code ='03'
and date_valid = '{dt}'
) ) as t2
where t2.Identifiant in (
sel distinct party_id
from ibfs_{i}_m_marketing_vu.VF_Dm_party_Detail
where  Date_Valid = '{dt}'
and Segmentation_Code_IBFS like '101%'
and party_status='C'
and mandatory_product_flag='Y'
and client_count_flag = 'Y'
and Residence_Country_Name like 'BENIN%'
)    
    ;"""
    liste_1.append(sql_string_i)

In [6]:
Data_Global = []
for i in liste_1:
    cursor = con.cursor() 
    Data_Global.append(pd.read_sql(i,con))  

In [7]:
Base_BEN_1 = Data_Global[0]

##### Etape des virement régulier

In [9]:
sql_string4 = f"""

----------- Information sur les transactions

select pt2.Pays, pt3.party_id,

--- flag des Statuts global
  sum(decode(to_char(pt2.tran_sign),'1',1,0)*tran_amt) mtn,
  sum(decode(to_char(pt2.tran_sign),'1',1,0)) nbr, 
  case when sum(decode(to_char(pt2.tran_sign),'1',1,0)*tran_amt)>80000 then 1 else 0 end as flag_mtn,

--- flag des Statuts 202404

sum(decode(pt2.month_id,'202404',1,0)*decode(to_char(pt2.tran_sign),'1',1,0)*tran_amt) mtn_202404, 
sum(decode(pt2.month_id,'202404',1,0)*decode(to_char(pt2.tran_sign),'1',1,0)) nbr_202404, 
case when sum(decode(pt2.month_id,'202404',1,0)*decode(to_char(pt2.tran_sign),'1',1,0)*tran_amt)>80000 then 1 else 0 end as flag_mtn_202404,

--- flag des Statuts 202402

sum(decode(pt2.month_id,'202402',1,0)*decode(to_char(pt2.tran_sign),'1',1,0)*tran_amt) mtn_202402, 
sum(decode(pt2.month_id,'202402',1,0)*decode(to_char(pt2.tran_sign),'1',1,0)) nbr_202402, 
case when sum(decode(pt2.month_id,'202402',1,0)*decode(to_char(pt2.tran_sign),'1',1,0)*tran_amt)>80000 then 1 else 0 end as flag_mtn_202402,

--- flag des Statuts 202403

sum(decode(pt2.month_id,'202403',1,0)*decode(to_char(pt2.tran_sign),'1',1,0)*tran_amt) mtn_202403, 
sum(decode(pt2.month_id,'202403',1,0)*decode(to_char(pt2.tran_sign),'1',1,0)) nbr_202403, 
case when sum(decode(pt2.month_id,'202403',1,0)*decode(to_char(pt2.tran_sign),'1',1,0)*tran_amt)>80000 then 1 else 0 end as flag_mtn_202403,

flag_mtn_202404+flag_mtn_202402+flag_mtn_202403 as flag_tot

from 
-------
  (
select 'SGBEN' Pays, to_char(cast(rai1.Effective_Date as date),'yyyymm') month_id, rai1.operation_code, rai1.Currency_code,rai1.tran_Amt, rai1.Account_Id,rai1.Tran_Desc,  rai1.tran_sign from ibfs_BEN_dw_vu.Posted_Transaction rai1 Where rai1.Tran_status in ('FO', 'VF', 'VA', 'XX') and
  to_char(cast(rai1.Effective_Date as date),'yyyymm') between to_char(add_months(to_date('202404','yyyymm'),-2),'yyyymm') and '202404'
 and to_char(rai1.tran_sign)='1' and 
rai1.operation_code in ('50','70','82','86','92','110','114','244','344','347','381','500','502','512','515','521','540','588','595','614','639','642','893','894','900','901')

) pt2
------------
  inner  join 

( 
(select 'SGBEN' Pays, jo1.Operation_Group_4_Code,jo1.Operation_Group_2_Code, jo1.operation_code from ibfs_BEN_rep.V_Operation_Group jo1) 
)
vog2 on pt2.operation_code = vog2.operation_code and pt2.pays= vog2.pays
------------
inner join 
(
(select 'SGBEN' Pays, to_char(cast( iza1.date_valid as date),'yyyymm') month_id,   iza1.rate_value,  iza1.currency2_code from  ibfs_BEN_dw_vu.dm_currency_rate  iza1
  where to_char(cast( iza1.date_valid as date),'yyyymm') between to_char(add_months(to_date('202404','yyyymm'),-2),'yyyymm') and '202404'
) 
) dcr1 
on pt2.Currency_code = dcr1.currency2_code and pt2.month_id=dcr1.month_id and pt2.pays= dcr1.pays
-----------------------------------
inner join 
( select ba1.* from
(
(select 'SGBEN' Pays,   hraz1.* from ibfs_BEN_dw_vu.Bank_Account  hraz1 ) 
)ba1
inner join 
(
(select 'SGBEN' Pays,  ja1.* from ibfs_BEN_dw_vu.Product  ja1 ) 
)prod1 

on prod1.Product_code = ba1.Product_code and prod1.pays=ba1.pays 

) pt3 on pt2.Account_Id = pt3.Account_Id and pt2.pays= pt3.pays
 
 where   
 (
pt2.tran_desc not like '%FRAIS%' or pt2.tran_desc not like '%FEES%' or 
pt2.tran_desc not like '%TIMBRE%' or pt2.tran_desc not like '%COMM%' or 
pt2.tran_desc not like '%CION%' or  pt2.tran_desc not like '%FRS%'
)
 group by pt2.pays, pt3.party_id
;""" 
cursor = con.cursor()
Virement_regulier = cursor.execute(sql_string4)
Virement_regulier = pd.read_sql(sql_string4, con)
Virement_regulier = Virement_regulier[['Party_Id','flag_tot']]
Virement_regulier = Virement_regulier[Virement_regulier['flag_tot'] >= 2]

##### OC1 : Crédit à échoir 
PPO 0-36 mois => échéance 3 moi et 
PPO 36 et + => échéance 6 mos


In [10]:
Virement_regulier = Virement_regulier[Virement_regulier['Party_Id'] != 'IA0000000']
Virement_regulier['Party_Id'] =Virement_regulier['Party_Id'].astype('int64')
Base_BEN_1['Identifiant'] = Base_BEN_1['Identifiant'].astype('int64')
OC1 = pd.merge(Base_BEN_1,Virement_regulier,left_on='Identifiant',right_on='Party_Id',how='inner')

In [11]:
OC1 = OC1[['Identifiant', 'Nom_client', 'Segmentation_Code_Description',
       'Profile_Code_Description', 'Code_gestionnaire', 'gestionnaire',
       'agence', 'age', 'Activity_Flag', 'Profession_Group_Desc_1',
       'Montant_du_Credit', 'Revenu', 'Contact_Text', 'Product_Name',
       'Contract_Date', 'Maturity_Date', 'type_']]

In [12]:
OC1.head()

,Identifiant,Nom_client,Segmentation_Code_Description,Profile_Code_Description,Code_gestionnaire,gestionnaire,agence,age,Activity_Flag,Profession_Group_Desc_1,Montant_du_Credit,Revenu,Contact_Text,Product_Name,Contract_Date,Maturity_Date,type_
0,101859,ADJIBABI WASSI,10104 - Clientèle patrimoniale,FONCTIONNAIRES TBB + 4%,440,AMADAH ALFA ROKIATH,COTONOU AGENCE PRINCIPALE,63.0,N,FONCTIONNAIRES,6480000.0,1633697.0,(+229)21300155; (+229)21337920; (+229)97885701,530120 - PPO MOYEN TERME - PPO MOYEN TERME,2021-11-12,2024-04-25,Credits echus et non renouvelles sur les 6 der...
1,101859,ADJIBABI WASSI,10104 - Clientèle patrimoniale,FONCTIONNAIRES TBB + 4%,440,AMADAH ALFA ROKIATH,COTONOU AGENCE PRINCIPALE,63.0,N,FONCTIONNAIRES,30000000.0,1633697.0,(+229)21300155; (+229)21337920; (+229)97885701,530120 - PPO MOYEN TERME - PPO MOYEN TERME,2020-08-04,2024-04-25,Credits echus et non renouvelles sur les 6 der...
2,104611,ADJINACOU GNAHOUI THIERRY CLAUDE SYLVAIN,10103 - Clientèle bonne gamme,SALARIES ALLIANZ BENIN TBB+4%,493,NOUROU MIKAILA,COTONOU AGENCE ST MICHEL,59.0,Y,EMPLOYES DU SECTEUR PRIVE,43800000.0,1483527.0,(+229)95068711; (+229)97045820,530120 - PPO MOYEN TERME - PPO MOYEN TERME,2017-06-06,2024-05-20,Credits arrivant a echeance dans – de 3 mois
3,107081,GNIGLA BLANCHE ARMELLE,10103 - Clientèle bonne gamme,PARTICULIERS AVEC TBB+4%,440,AMADAH ALFA ROKIATH,COTONOU AGENCE PRINCIPALE,48.0,Y,EMPLOYES DU SECTEUR PRIVE,19000000.0,571503.0,(+229)21314081; (+229)95565555,555122 - FGS MOYEN TERME - EXPRESSO MOYEN TER...,2018-08-06,2024-07-20,Credits arrivant a echeance dans – de 3 mois
4,200249,AGBOGBA CESAIRE MAHOULE,10103 - Clientèle bonne gamme,SALARIES AHS TBB+4,440,AMADAH ALFA ROKIATH,COTONOU AGENCE PRINCIPALE,57.0,Y,EMPLOYES DU SECTEUR PRIVE,10000000.0,729548.0,(+229)21333898; (+229)95564515; (+229)97972204,555122 - FGS MOYEN TERME - EXPRESSO MOYEN TER...,2019-07-11,2024-07-20,Credits arrivant a echeance dans – de 3 mois


In [13]:
chemin = '/home/e_ekoua2/BISME/data/Dossier_JC/BEN/'
dfic = glob.glob(chemin + '/OC1_*.xlsx')
dfs=[]
for i in dfic:
    df = pd.read_excel(i)
    dfs.append(df)
anciens_OC1 = pd.concat(dfs,ignore_index = True)

In [105]:
import os
current_dir = os.getcwd()
 
print(f"JupyterLab currently in folder {current_dir}")

JupyterLab currently in folder /home/e_ekoua2/BISME/data/Dossier_JC


In [14]:
 anciens_OC1 = anciens_OC1[['Identifiant']]

In [15]:
OC1.shape

(242, 17)

In [16]:
OC1 = OC1[~OC1['Identifiant'].isin(anciens_OC1['Identifiant'])]

In [17]:
def get_clients_1(df):
    result = pd.DataFrame(columns = df.columns)
    for _, group in df.groupby('Code_gestionnaire'):
        group_1 = group[group['type_'] == 'Credits echus et non renouvelles sur les 6 derniers mois'].head(2)
        group_2 = group[group['type_'] == 'Credits arrivant a echeance dans – de 3 mois'].head(2)
        combined_group = pd.concat([group_1,group_2],ignore_index = True)
        result = pd.concat([result,combined_group])
    return result
result = get_clients_1(OC1)       

In [18]:
nom_fichier_OC1 = f"{chemin}OC1_{num}.xlsx"
OC1.to_excel(nom_fichier_OC1)

##### OC3 : CSM créditeur depuis 2 et 3 mois

In [19]:
sql_string5 = f"""

select distinct a.party_id as Identifiant , a.party_short_name as Nom_client,a.segmentation_code_description ,Attrition_Score_Pct as score_attrition,
a.profile_code_description,a.officer_id as Code_gestionnaire,a.officer_name as gestionnaire,a.org_unit_name_4 as agence,
to_number(substr(a.Age_Group_Description,1,2)) as age,a.activity_flag,profession_desc,a.marketing_income_amt as Revenu,
a.contact_text,'CSM créditeur depuis 2 et 3 mois' as type_
from ibfs_BEN_m_marketing_vu.VF_Dm_party_Detail a
where  Date_Valid = '{dt}'
and Segmentation_Code_IBFS like '101%'
and party_status='C'
and not (a.profile_code_description like  '%PERSONNEL%'  or a.profile_code_description like  '%DECEDE%')
and mandatory_product_flag='Y'
and client_count_flag = 'Y'
and party_id in
(select party_id  from (

select rr.party_id, max(rr.flag_initiative) flag_initiative, max(rr.flag_Non_initiative) flag_Non_initiative from

(
select pt2.Pays as Pays_tran,pt2.Effective_Date,  pt3.party_id as party_id, pt3.product_code, vog2.operation_code,tran_Amt,tran_sign,
vog2.Operation_Type_Desc,vog2.Operation_Type_Label,vog2.Operation_Group_4_Name,
Nvl(vog2.flag_initiative,0) flag_initiative, Nvl(vog2.flag_Non_initiative,0) flag_Non_initiative

from 
(
select 'SGBEN' Pays,  rai10.Effective_Date, rai10.operation_code, rai10.tran_Amt, rai10.Account_Id,rai10.Tran_Desc,  
rai10.tran_sign from ibfs_BEN_dw_vu.Posted_Transaction rai10 Where rai10.Tran_status in
('FO', 'VF', 'VA', 'XX') and  rai10.Effective_Date between add_months('{dt}', -2) and  '{dt}'
) pt2
  inner  join 
( 
select 'SGBEN' Pays,jo10.operation_code,jo10.Operation_Type_Desc, jo10.Operation_Type_Label,jo10.Operation_Group_4_Name, dd.param_value,

case when dd.param_value is null then 0 else 1 end as  flag_initiative,

case when dd.param_value is null then 1 else 0 end as flag_Non_initiative

from ibfs_BEN_rep.V_Operation_Group jo10 left join 
ibfs_BEN_dw_vu.R_categorization_params dd on dd.param_value=jo10.operation_code
)
vog2 on pt2.operation_code = vog2.operation_code and pt2.pays= vog2.pays
------------
inner join 
( 
select 'SGBEN' Pays,   hraz10.* from ibfs_BEN_dw_vu.Bank_Account  hraz10 
) pt3 on pt2.Account_Id = pt3.Account_Id and pt2.pays= pt3.pays

) rr


group by rr.party_id
) rd where rd.flag_Non_initiative=1 and rd.flag_initiative=0)

;""" 
cursor = con.cursor()
OC3 = cursor.execute(sql_string5)
OC3 = result_Set5 = pd.read_sql(sql_string5, con)

In [21]:
chemin = '/home/e_ekoua2/BISME/data/Dossier_JC/BEN/'
dfic = glob.glob(chemin + '/OC3_*.xlsx')
dfs=[]
for i in dfic:
    df = pd.read_excel(i)
    dfs.append(df)
anciens_OC3 = pd.concat(dfs,ignore_index = True)

In [22]:
anciens_OC3 = anciens_OC3[['Identifiant']]

In [23]:
OC3 = OC3[~OC3['Identifiant'].isin(anciens_OC3['Identifiant'])]

In [24]:
def get_clients_2(df):
    results = pd.DataFrame(columns = df.columns)
    for _, group in df.groupby('Code_gestionnaire'):
        group_1 = group[group['type_'] == 'CSM créditeur depuis 2 et 3 mois'].head(2)
        combined_group = pd.concat([group_1],ignore_index = True)
        results = pd.concat([results,combined_group])
    return results
results = get_clients_2(OC3) 
OC3_ = results.copy()

In [25]:
nom_fichier_OC3 = f"{chemin}OC3_{num}.xlsx"
OC3_.to_excel(nom_fichier_OC3)

#####  OC4 : Clients sans PPO avec VRT salaire et ancienneté de moins de 3ans  et  OC5: Clients sans PPO avec VRT salaire et ancienneté de plus  de 3ans 

In [26]:
sql_string6 = f"""

select t3.* 
from 
(select t2.*
from 
(select t1.* 
from 
(select distinct a.party_id as Identifiant , a.party_short_name as Nom_client,a.segmentation_code_description ,Appetance_Score_PPO_Pct as Appetance_ppo,
a.profile_code_description,a.officer_id as Code_gestionnaire,a.officer_name as gestionnaire,a.org_unit_name_4 as agence,
to_number(substr(a.Age_Group_Description,1,2)) as age,a.activity_flag,profession_desc,a.marketing_income_amt as Revenu,
a.contact_text,case when Relationship_Duration_Months<36 then 'Clients sans PPO avec anciennete < 3 ans' else 'Clients sans PPO avec anciennete > 3 ans' end as type_
from ibfs_BEN_m_marketing_vu.VF_Dm_party_Detail a
where  Date_Valid = '{dt}'
and Segmentation_Code_IBFS like '101%'
and party_status= 'C'
and age between 22 and 64
and Appetance_ppo > 0.5
and revenu > 80000
and mandatory_product_flag='Y'
and client_count_flag = 'Y' 
and  party_id in (
select distinct a.party_id 
from ibfs_BEN_m_marketing_vu.vf_dd_agreement_detail a
inner join ibfs_BEN_rep.V_C_Product_MKTRE b
on a.product_code = b.product_code
where a.date_valid = '{dt}'
and  a.mandatory_product_flag = 'Y'
and a.Segmentation_Code_IBFS like '101%'
and a.party_status = 'C'
and a.product_count_flag = 'Y'
and not (a.profile_code_description like  '%PERSONNEL%'  or a.profile_code_description like  '%DECEDE%')
and b.product_group_code_2 = '1RECC')) as t1
where t1.Identifiant not in (
select distinct party_id 
from  ibfs_ben_m_risk_vu.dm_Agreement 
where Provi_Stage_Code ='03'
and date_valid = '{dt}'
)) as t2
where t2.Identifiant not in (

select distinct party_id 
from ibfs_BEN_m_marketing_vu.vf_dd_agreement_detail
where date_valid = '{dd}'
and  mandatory_product_flag = 'Y'
and Segmentation_Code_IBFS like '101%'
and party_status = 'C'
and product_count_flag = 'Y'
and not (profile_code_description like  '%PERSONNEL%'  or profile_code_description like  '%DECEDE%')
and Product_Group_Code_3 like '1RECRCC%'

)) as t3

;""" 
cursor = con.cursor()
OC4_OC5_1 = cursor.execute(sql_string6)
OC4_OC5_1 = pd.read_sql(sql_string6, con)

In [27]:
OC4_OC5_1['Identifiant'] = OC4_OC5_1['Identifiant'].astype('int64') 
OC4_OC5 = OC4_OC5_1.merge(Virement_regulier,left_on='Identifiant',right_on='Party_Id',how='inner')

In [28]:
OC4_OC5 = OC4_OC5[['Identifiant', 'Nom_client', 'Segmentation_Code_Description',
       'Appetance_ppo', 'Profile_Code_Description', 'Code_gestionnaire',
       'gestionnaire', 'agence', 'age', 'Activity_Flag', 'Profession_Desc',
       'Revenu', 'Contact_Text', 'type_']]

In [29]:
OC4_OC5.shape

(2582, 14)

##### OC6 : Clients sans ACT ou en découvert avec VRT salaire 

In [30]:
sql_string7 = f"""


select t3.* 
from 
(select t2.*
from 
(select t1.* 
from 
(select distinct a.party_id as Identifiant , a.party_short_name as Nom_client,a.segmentation_code_description ,Appetance_Score_PPO_Pct as Appetance_ppo,
a.profile_code_description,a.officer_id as Code_gestionnaire,a.officer_name as gestionnaire,a.org_unit_name_4 as agence,
to_number(substr(a.Age_Group_Description,1,2)) as age,a.activity_flag,profession_desc,a.marketing_income_amt as Revenu,
a.contact_text,'OC6 client_Sans_ACT avec VRT régulier' as type_
from ibfs_BEN_m_marketing_vu.VF_Dm_party_Detail a
where  Date_Valid = '{dt}'
and Segmentation_Code_IBFS like '101%'
and party_status= 'C'
and age between 22 and 64
and Appetance_ppo > 0.5
and revenu > 80000
and mandatory_product_flag='Y'
and client_count_flag = 'Y' 
and  party_id in (
select distinct a.party_id 
from ibfs_BEN_m_marketing_vu.vf_dd_agreement_detail a
inner join ibfs_BEN_rep.V_C_Product_MKTRE b
on a.product_code = b.product_code
where a.date_valid = '{dt}'
and  a.mandatory_product_flag = 'Y'
and a.Segmentation_Code_IBFS like '101%'
and a.party_status = 'C'
and a.product_count_flag = 'Y'
and not (a.profile_code_description like  '%PERSONNEL%'  or a.profile_code_description like  '%DECEDE%')
and b.product_group_code_2 = '1RECC')) as t1
where t1.Identifiant not in (
select distinct party_id 
from  ibfs_ben_m_risk_vu.dm_Agreement 
where Provi_Stage_Code ='03'
and date_valid = '{dt}'
)) as t2
where t2.Identifiant not in (

select distinct party_id 
from ibfs_BEN_m_marketing_vu.vf_dd_agreement_detail
where date_valid = '{dd}'
and  mandatory_product_flag = 'Y'
and Segmentation_Code_IBFS like '101%'
and party_status = 'C'
and product_count_flag = 'Y'
and not (profile_code_description like  '%PERSONNEL%'  or profile_code_description like  '%DECEDE%')
and  Product_Group_Code_3 like '1RECRFC%'
)) as t3

;""" 
cursor = con.cursor()
OC6_ = cursor.execute(sql_string7)
OC6_ = pd.read_sql(sql_string7, con)

In [31]:
OC6_['Identifiant'] = OC6_['Identifiant'].astype('int64')
OC6 = OC6_.merge(Virement_regulier,left_on='Identifiant',right_on='Party_Id',how='inner')

In [32]:
OC6 = OC6[['Identifiant', 'Nom_client', 'Segmentation_Code_Description',
       'Appetance_ppo', 'Profile_Code_Description', 'Code_gestionnaire',
       'gestionnaire', 'agence', 'age', 'Activity_Flag', 'Profession_Desc',
       'Revenu', 'Contact_Text', 'type_']]

##### OC7 : Clients sans Package ou en découvert avec VRT salaire 

In [33]:
sql_string8 = f"""
select t3.* 
from 
(select t2.*
from 
(select t1.* 
from 
(select distinct a.party_id as Identifiant , a.party_short_name as Nom_client,a.segmentation_code_description ,Appetance_Score_PPO_Pct as Appetance_ppo,
a.profile_code_description,a.officer_id as Code_gestionnaire,a.officer_name as gestionnaire,a.org_unit_name_4 as agence,
to_number(substr(a.Age_Group_Description,1,2)) as age,a.activity_flag,profession_desc,a.marketing_income_amt as Revenu,
a.contact_text,'OC7 client_Sans_Package avec VRT régulier' as type_
from ibfs_BEN_m_marketing_vu.VF_Dm_party_Detail a
where  Date_Valid = '{dt}'
and Segmentation_Code_IBFS like '101%'
and party_status= 'C'
and age between 22 and 64
and Appetance_ppo > 0.5
and revenu > 80000
and mandatory_product_flag='Y'
and client_count_flag = 'Y' 
and  party_id in (
select distinct a.party_id 
from ibfs_BEN_m_marketing_vu.vf_dd_agreement_detail a
inner join ibfs_BEN_rep.V_C_Product_MKTRE b
on a.product_code = b.product_code
where a.date_valid = '{dt}'
and  a.mandatory_product_flag = 'Y'
and a.Segmentation_Code_IBFS like '101%'
and a.party_status = 'C'
and a.product_count_flag = 'Y'
and not (a.profile_code_description like  '%PERSONNEL%'  or a.profile_code_description like  '%DECEDE%')
and b.product_group_code_2 = '1RECC')) as t1
where t1.Identifiant not in (
select distinct party_id 
from  ibfs_ben_m_risk_vu.dm_Agreement 
where Provi_Stage_Code ='03'
and date_valid = '{dt}'
)) as t2
where t2.Identifiant not in (

select  distinct party_id 
from ibfs_BEN_m_marketing_vu.vf_Dm_agreement_Detail
where org_structure_type = 'C'
and party_status = 'C'
and mandatory_product_flag = 'Y'
and segmentation_code_ibfs like '101%'
and date_valid = '{dt}'
and product_count_flag = 'Y'
and product_status = 'Y'
and package_count_flag ='Y'
)) as t3
;""" 
cursor = con.cursor()
OC7_ = cursor.execute(sql_string8)
OC7_ = pd.read_sql(sql_string8, con)

In [34]:
OC7_['Identifiant'] = OC7_['Identifiant'].astype('int64')
OC7 = OC7_.merge(Virement_regulier,left_on='Identifiant',right_on='Party_Id',how='inner')

In [35]:
OC7 = OC7[['Identifiant', 'Nom_client', 'Segmentation_Code_Description',
       'Appetance_ppo', 'Profile_Code_Description', 'Code_gestionnaire',
       'gestionnaire', 'agence', 'age', 'Activity_Flag', 'Profession_Desc',
       'Revenu', 'Contact_Text', 'type_']]

In [40]:
Bases_OC_Total = pd.concat([OC4_OC5,OC6,OC7],ignore_index = True)

In [41]:
Bases_OC_Total['cle'] = Bases_OC_Total['Identifiant'].astype(str) + '_' + Bases_OC_Total['type_'].astype(str)

In [43]:
chemin = '/home/e_ekoua2/BISME/data/Dossier_JC/BEN/'
dfic = glob.glob(chemin + '/Bases_OC_Total_*.xlsx')
dfs=[]
for i in dfic:
    df = pd.read_excel(i)
    dfs.append(df)
anciens_Bases_OC_Total = pd.concat(dfs,ignore_index = True)

In [46]:
anciens_Bases_OC_Total = anciens_Bases_OC_Total[['Identifiant', 'Nom_client', 'Segmentation_Code_Description',
       'Appetance_ppo', 'Profile_Code_Description', 'Code_gestionnaire',
       'gestionnaire', 'agence', 'age', 'Activity_Flag', 'Profession_Desc',
       'Revenu', 'Contact_Text', 'type_']]

In [47]:
anciens_Bases_OC_Total['cle'] = anciens_Bases_OC_Total['Identifiant'].astype(str) + '_' + anciens_Bases_OC_Total['type_'].astype(str)

In [48]:
Bases_OC_Total = Bases_OC_Total[~Bases_OC_Total['cle'].isin(anciens_Bases_OC_Total['cle'])]

In [49]:
Bases_OC_Total = Bases_OC_Total[['Identifiant', 'Nom_client', 'Segmentation_Code_Description',
       'Appetance_ppo', 'Profile_Code_Description', 'Code_gestionnaire',
       'gestionnaire', 'agence', 'age', 'Activity_Flag', 'Profession_Desc',
       'Revenu', 'Contact_Text', 'type_']]

In [50]:
def get_clients_3(df):
    results = pd.DataFrame(columns = df.columns)
    for _, group in df.groupby('Code_gestionnaire'):
        group_1 = group[group['type_'] == 'Clients sans PPO avec anciennete < 3 ans'].head(5)
        group_2 = group[group['type_'] == 'Clients sans PPO avec anciennete > 3 ans'].head(5)
        group_3 = group[group['type_'] == 'OC6 client_Sans_ACT avec VRT régulier'].head(2)
        group_4 = group[group['type_'] == 'OC7 client_Sans_Package avec VRT régulier'].head(2)
        combined_group = pd.concat([group_1,group_2,group_3,group_4],ignore_index = True)
        results = pd.concat([results,combined_group])
    return results
results = get_clients_3(Bases_OC_Total) 
Bases_OC_Total_ = results.copy()

In [36]:
nom_fichier_new = f"{chemin}Bases_OC_Total_{num}.xlsx"
Bases_OC_Total_.to_excel(nom_fichier_new)

In [51]:
Bases_OC_Total['type_'].unique()

array(['Clients sans PPO avec anciennete > 3 ans',
       'Clients sans PPO avec anciennete < 3 ans',
       'OC6 client_Sans_ACT avec VRT régulier',
       'OC7 client_Sans_Package avec VRT régulier'], dtype=object)

In [109]:
Bases_OC_Total_.shape 

(426, 14)

In [123]:
import smtplib

In [124]:
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

def envoyer_email(destinataire,sujet,corps_message,piece_jointe,expediteur,mot_de_passe,serveur_smtp,port_smtp):
    message = MIMEMultipart()
    message['from'] = expediteur
    message['To'] = destinataire
    message['Subject'] = sujet
    message.attach(MIMEText(corps_message,'plain'))

    nom_piece_jointe = piece_jointe.split('/')[-1]
    piece_jointe_mime = MIMBase('application','octet-stream')
    piece_jointe_mime.set_payload(open(piece_jointe,'rb').read())
    encoders.encode_base64(piece_jointe_mime)
    piece_jointe_mime.add_header('content-Disposition',f'attachament; filename = {nom_piece_jointe}')
    message.attach(piece_jointe_mime)

    serveur = smtplib.SMTP(serveur_smtp,port_smtp)
    serveur.starttls()
    serveur.login(expediteur,mot_de_passe)
    serveur.sendmail(expediteur,destinataire,message.as_string())
    serveur.quit()

destinataire = 'exemple@hdh.com'
sujet = 'fichier Excel'
corps = 'Hello , veillez trouvez le fichier'
piece_jointe = chemin
expediteur = 'expediteur@hej.com'
mot_de_passe = 'frtehdj'
serveur_smtp = 'smtp.exempale.com'
port_smtp = 589
    
    